<a href="https://colab.research.google.com/github/Asigen93/deteksiayam_yolov8_and_deepsort/blob/main/beta2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# jalankan di Colab cell (prefix !)
!pip install -U pip
!pip install ultralytics roboflow opencv-python-headless matplotlib tqdm pandas
# TensorFlow dibutuhkan untuk export/validasi TFLite
!pip install -U "tensorflow>=2.11.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 82.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 105.7 MB/s  0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [ultralytics]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 443.3/

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="QIxsCi9aE9bKIfs7vXkg")  # paste API key Anda
# Ganti workspace/project/version sesuai Roboflow Anda:
# Anda bisa dapatkan nama project & versi dari URL Roboflow atau dari dashboard
project = rf.workspace().project("countingayam-om39k-jj99v")  # jika error, lihat catatan di bawah
dataset = project.version(1).download("yolov8")  # pilihan export: "yolov8" atau "yolov5"
print("Dataset downloaded to:", dataset.location)

In [ ]:
import os
# sesuaikan path kalau Anda pakai dataset.location dari SDK, contohnya:
dataset_path = "/content/countingayam-1"   # ganti sesuai lokasi unzip / dataset.location
# kalau pakai SDK: dataset_path = dataset.location

for root, dirs, files in os.walk(dataset_path):
    print(root, "->", len([f for f in files if f.endswith(('.jpg','.png'))]), "images")
    # break supaya tidak print semua
    break

# Jika Roboflow sudah menyertakan data.yaml, gunakan itu.
!sed -n '1,200p' {dataset_path}/data.yaml

In [ ]:
print(dataset.location)

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.yaml")
model.train(data="countingayam-1/data.yaml", epochs=50)

In [ ]:
# Validasi
results = model.val(data="/content/countingayam-1/data.yaml")
print(results)

# Prediksi contoh gambar di val/images
model.predict(
    source="/content/countingayam-1/valid/images",
    save=True,
    conf=0.25,
    imgsz=640
)

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/confusion_matrix.png"))

In [ ]:
from IPython.display import Image, display

# Tampilkan confusion matrix
display(Image(filename="/content/runs/detect/train/results.png"))

In [ ]:
import cv2, numpy as np, os
from ultralytics import YOLO
from collections import OrderedDict

# --- CentroidTracker (sederhana) ---
class CentroidTracker:
    def __init__(self, max_disappeared=30, max_distance=50):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        self.counted_ids = set()
        self.history = {}  # history centroids per id

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.history[self.nextObjectID] = [centroid]
        self.nextObjectID += 1

    def deregister(self, objectID):
        if objectID in self.objects: del self.objects[objectID]
        if objectID in self.disappeared: del self.disappeared[objectID]
        if objectID in self.history: del self.history[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objID in list(self.disappeared.keys()):
                self.disappeared[objID] += 1
                if self.disappeared[objID] > self.max_disappeared:
                    self.deregister(objID)
            return self.objects

        input_centroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (x1,y1,x2,y2)) in enumerate(rects):
            input_centroids[i] = (int((x1+x2)/2.0), int((y1+y2)/2.0))

        if len(self.objects) == 0:
            for i in range(len(input_centroids)):
                self.register(input_centroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            D = np.linalg.norm(np.array(objectCentroids)[:, np.newaxis] - input_centroids[np.newaxis, :], axis=2)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows, usedCols = set(), set()
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols: continue
                if D[row, col] > self.max_distance: continue
                objectID = objectIDs[row]
                self.objects[objectID] = input_centroids[col]
                self.disappeared[objectID] = 0
                self.history[objectID].append(tuple(input_centroids[col]))
                usedRows.add(row); usedCols.add(col)
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.max_disappeared:
                    self.deregister(objectID)
            for col in unusedCols:
                self.register(input_centroids[col])
        return self.objects

# --- inference + counting function ---
def infer_and_count_video(video_path, weight_path, labels=['ayam'], out_video='out_count.mp4', conf_thres=0.25, iou_thres=0.45):
    model = YOLO(weight_path)
    cap = cv2.VideoCapture(video_path)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_video, fourcc, fps, (w,h))

    tracker = CentroidTracker(max_disappeared=30, max_distance=60)
    total_count = 0

    # garis vertikal di tengah frame
    line_x = int(w * 0.5)

    while True:
        ret, frame = cap.read()
        if not ret: break

        results = model.predict(frame, imgsz=960, conf=conf_thres, iou=iou_thres, verbose=False)[0]
        boxes, classes, confs = [], [], []
        for r in results.boxes:
            cls = int(r.cls.cpu().numpy()[0]) if hasattr(r,'cls') else int(r.cls)
            conf = float(r.conf.cpu().numpy()[0]) if hasattr(r,'conf') else float(r.conf)
            x1,y1,x2,y2 = r.xyxy[0].cpu().numpy()
            boxes.append([int(x1),int(y1),int(x2),int(y2)])
            classes.append(labels[cls] if cls < len(labels) else str(cls))
            confs.append(conf)

        tracked = tracker.update(boxes)

        # mapping each box ke nearest tracked ID
        object_ids = []
        centroids_list = [((b[0]+b[2])//2, (b[1]+b[3])//2) for b in boxes]
        for c in centroids_list:
            min_id = None; min_dist = None
            for oid, ocent in tracked.items():
                d = np.linalg.norm(np.array(ocent) - np.array(c))
                if min_dist is None or d < min_dist:
                    min_dist = d; min_id = oid
            object_ids.append(min_id)

        # --- line-crossing robust (kiri <-> kanan) ---
        for idx, oid in enumerate(object_ids):
            if oid is None: continue
            prev = tracker.history.get(oid, [])
            if len(prev) >= 2:
                prev_x = prev[-2][0]
                curr_x = prev[-1][0]
                if prev_x < line_x and curr_x >= line_x and oid not in tracker.counted_ids:
                    tracker.counted_ids.add(oid)
                    total_count += 1
                elif prev_x > line_x and curr_x <= line_x and oid not in tracker.counted_ids:
                    tracker.counted_ids.add(oid)
                    total_count += 1

        # draw boxes & display ID
        for i,box in enumerate(boxes):
            x1,y1,x2,y2 = box
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
            txt = f"{classes[i]} {confs[i]:.2f} ID:{object_ids[i]}"
            cv2.putText(frame, txt, (x1, y1-15), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 3)

        # garis vertikal
        cv2.line(frame, (line_x,0), (line_x,h), (255,0,0), 3)
        cv2.putText(frame, f"Total count: {total_count}", (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 4)

        writer.write(frame)

    cap.release()
    writer.release()
    print("Selesai. Output:", out_video)


# --- contoh panggil ---
infer_and_count_video(
    '/content/tes1.mp4',
    'runs/detect/train/weights/best.pt',
    labels=['ayam'],
    out_video='result_count.mp4',
    conf_thres=0.25,
    iou_thres=0.3   # lebih rendah supaya pisah bounding box
)

In [ ]:
from ultralytics import YOLO

# path weight hasil training:
weight_path = "runs/detect/train/weights/best.pt"
model = YOLO(weight_path)

# 1) Export FP16 (smaller, default no int8):
model.export(format="tflite", imgsz=640, optimize=True)
# Hasil: file seperti best-fp16.tflite di working dir

# 2) (Opsional) Export INT8 (post-training quantization)
# Pastikan dataset/data.yaml ada, Ultralytics akan gunakan data untuk calibrasi quantization
try:
    model.export(format="tflite", imgsz=640, int8=True, data='data.yaml', optimize=True)
    print("INT8 export selesai.")
except Exception as e:
    print("INT8 export gagal, cek error:", e)
    print("Gunakan fallback FP16 atau lakukan quantization manual.")

In [ ]:
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="best-fp16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input:", input_details)
print("Output:", output_details)

# contoh preprocess satu image:
import cv2, numpy as np
img = cv2.imread('/content/some_test_image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = input_details[0]['shape'][1], input_details[0]['shape'][2]
img_resized = cv2.resize(img, (w,h))
img_arr = img_resized.astype('float32') / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

interpreter.set_tensor(input_details[0]['index'], img_arr)
interpreter.invoke()
outs = [interpreter.get_tensor(o['index']) for o in output_details]
for o in outs:
    print("out shape:", o.shape)
# inspeksi array untuk tahu format keluaran; parsing custom diperlukan